First, import needed modules

In [178]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point, LineString, Polygon
import shapely.speedups
from geopy.geocoders import Nominatim
import os

Initalize variables

In [179]:
path = os.getcwd()
path

'/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters'

Load Dataset which has the endpoints of power line projects

In [180]:
powerlines = pd.read_csv(f'{path}/data/geodata/powerlines.csv', sep=';')
#powerlines = powerlines.set_index("# BBPlG 2013", drop = False)
powerlines.head()

,# BBPlG 2013,Start,End,New,Retro
0,1,Emden-Borssum,Osterath,1.0,NaN
1,2,Osterath,Philippsburg,1.0,NaN
2,3,Brunsbüttel,Großgartach,1.0,NaN
3,4,Wilster,Grafenrheinfeld,1.0,NaN
4,5,Bad Lauchstädt,Meitingen,1.0,NaN


Get the geocoordinates of the 'endpoint' cities using OSM/Nominatim API

In [181]:
start_locations = geocode(powerlines['Start'], provider='nominatim', user_agent='plm', timeout=4)
start_locations

,geometry,address
0,POINT (7.22429 53.34982),"Emden-Borssum, Birkenweg, Borssum, Emden, Nied..."
1,POINT (6.62072 51.26909),"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh..."
2,POINT (9.13954 53.89725),"Brunsbüttel, Dithmarschen, Schleswig-Holstein,..."
3,POINT (9.37338 53.92328),"Wilster, Kreis Steinburg, Schleswig-Holstein, ..."
4,POINT (11.86870 51.38813),"Bad Lauchstädt, Lindenstraße, Lindenstraße, Kl..."
...,...,...
56,POINT (9.07063 54.08988),"Meldorf, Mitteldithmarschen, Dithmarschen, Sch..."
57,POINT (7.20583 53.36705),"Emden, Niedersachsen, 26721, Deutschland"
58,POINT (7.16475 49.64882),"Birkenfeld, Landkreis Birkenfeld, Rheinland-Pf..."
59,POINT (10.20124 47.92271),"Woringen, Bad Grönenbach, Landkreis Unterallgä..."


In [182]:
end_locations = geocode(powerlines['End'], provider='nominatim', user_agent='plm', timeout=4)
end_locations

,geometry,address
0,POINT (6.62072 51.26909),"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh..."
1,POINT (8.45467 49.23675),"Philippsburg, Verwaltungsverband Philippsburg,..."
2,POINT (9.12587 49.14254),"Großgartach, Leingarten, Landkreis Heilbronn, ..."
3,POINT (10.19668 50.00553),"Grafenrheinfeld, Landkreis Schweinfurt, Bayern..."
4,POINT (10.85000 48.55000),"Meitingen, Landkreis Augsburg, Bayern, 86405, ..."
...,...,...
56,POINT (8.85870 54.12985),"Büsum, Büsum-Wesselburen, Dithmarschen, Schles..."
57,POINT (8.05776 53.32577),"Conneforde, Wiefelstede, Landkreis Ammerland, ..."
58,POINT (8.84929 48.94873),"Mühlacker, VVG der Stadt Mühlacker, Enzkreis, ..."
59,POINT (10.18132 47.98677),"Memmingen, Bayern, 87700, Deutschland"


Create powerlines_geo dataset by merging start and endpoints with the powerlines df

In [183]:
powerlines_geo_start = start_locations.join(powerlines, how = 'left')
powerlines_geo_start.rename(columns = {'geometry':'Start_geometry', 'address':'Start_address'}, inplace = True)
powerlines_geo = end_locations.join(powerlines_geo_start, how = 'left')
powerlines_geo.rename(columns = {'geometry':'End_geometry', 'address':'End_address'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['# BBPlG 2013', 'Start', 'Start_address', 'Start_geometry', 'End', 'End_address', 'End_geometry', 'direct_line', 'New', 'Retro'])
powerlines_geo

,# BBPlG 2013,Start,Start_address,Start_geometry,End,End_address,End_geometry,direct_line,New,Retro
0,1,Emden-Borssum,"Emden-Borssum, Birkenweg, Borssum, Emden, Nied...",POINT (7.22429 53.34982),Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),NaN,1.0,NaN
1,2,Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),Philippsburg,"Philippsburg, Verwaltungsverband Philippsburg,...",POINT (8.45467 49.23675),NaN,1.0,NaN
2,3,Brunsbüttel,"Brunsbüttel, Dithmarschen, Schleswig-Holstein,...",POINT (9.13954 53.89725),Großgartach,"Großgartach, Leingarten, Landkreis Heilbronn, ...",POINT (9.12587 49.14254),NaN,1.0,NaN
3,4,Wilster,"Wilster, Kreis Steinburg, Schleswig-Holstein, ...",POINT (9.37338 53.92328),Grafenrheinfeld,"Grafenrheinfeld, Landkreis Schweinfurt, Bayern...",POINT (10.19668 50.00553),NaN,1.0,NaN
4,5,Bad Lauchstädt,"Bad Lauchstädt, Lindenstraße, Lindenstraße, Kl...",POINT (11.86870 51.38813),Meitingen,"Meitingen, Landkreis Augsburg, Bayern, 86405, ...",POINT (10.85000 48.55000),NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
56,33,Meldorf,"Meldorf, Mitteldithmarschen, Dithmarschen, Sch...",POINT (9.07063 54.08988),Büsum,"Büsum, Büsum-Wesselburen, Dithmarschen, Schles...",POINT (8.85870 54.12985),NaN,1.0,NaN
57,34,Emden,"Emden, Niedersachsen, 26721, Deutschland",POINT (7.20583 53.36705),Conneforde,"Conneforde, Wiefelstede, Landkreis Ammerland, ...",POINT (8.05776 53.32577),NaN,NaN,1.0
58,35,Birkenfeld,"Birkenfeld, Landkreis Birkenfeld, Rheinland-Pf...",POINT (7.16475 49.64882),Mühlacker,"Mühlacker, VVG der Stadt Mühlacker, Enzkreis, ...",POINT (8.84929 48.94873),NaN,1.0,NaN
59,36,Woringen,"Woringen, Bad Grönenbach, Landkreis Unterallgä...",POINT (10.20124 47.92271),Memmingen,"Memmingen, Bayern, 87700, Deutschland",POINT (10.18132 47.98677),NaN,NaN,1.0


Create shapefiles from the start and endpoints dfs

In [184]:
powerlines_geo_end = end_locations.join(powerlines, how = 'left')
powerlines_geo_end.to_file(f'{path}/data/geodata/endpoints.shp')
powerlines_geo_start.rename(columns = {'Start_geometry':'geometry', 'Start_address':'address'}, inplace = True)
powerlines_geo_start.to_file(f'{path}/data/geodata/startpoints.shp')

/opt/miniconda3/envs/vscode/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_24746/2621956504.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  powerlines_geo_end.to_file(f'{path}/data/geodata/endpoints.shp')
/opt/miniconda3/envs/vscode/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_24746/2621956504.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  powerlines_geo_start.to_file(f'{path}/data/geodata/startpoint

Create direct line from start and end coordinates

In [185]:
for i in range (0, len(powerlines_geo)):
    start = powerlines_geo['Start_geometry'].loc[powerlines_geo.index[i]]
    end = powerlines_geo['End_geometry'].loc[powerlines_geo.index[i]]
    powerlines_geo.loc[powerlines_geo.index[i], 'direct_line'] = LineString([start, end])
powerlines_geo.head()

/opt/miniconda3/envs/vscode/lib/python3.10/site-packages/pandas/core/internals/blocks.py:930: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return self.coerce_to_target_dtype(value).setitem(indexer, value)
/opt/miniconda3/envs/vscode/lib/python3.10/site-packages/pandas/core/internals/managers.py:304: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  applied = getattr(b, f)(**kwargs)


,# BBPlG 2013,Start,Start_address,Start_geometry,End,End_address,End_geometry,direct_line,New,Retro
0,1,Emden-Borssum,"Emden-Borssum, Birkenweg, Borssum, Emden, Nied...",POINT (7.22429 53.34982),Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),"LINESTRING (7.22428778316919 53.349817, 6.6207...",1.0,NaN
1,2,Osterath,"Osterath, Meerbusch, Rhein-Kreis Neuss, Nordrh...",POINT (6.62072 51.26909),Philippsburg,"Philippsburg, Verwaltungsverband Philippsburg,...",POINT (8.45467 49.23675),"LINESTRING (6.6207151 51.2690922, 8.4546679 49...",1.0,NaN
2,3,Brunsbüttel,"Brunsbüttel, Dithmarschen, Schleswig-Holstein,...",POINT (9.13954 53.89725),Großgartach,"Großgartach, Leingarten, Landkreis Heilbronn, ...",POINT (9.12587 49.14254),"LINESTRING (9.1395423 53.8972549, 9.1258701 49...",1.0,NaN
3,4,Wilster,"Wilster, Kreis Steinburg, Schleswig-Holstein, ...",POINT (9.37338 53.92328),Grafenrheinfeld,"Grafenrheinfeld, Landkreis Schweinfurt, Bayern...",POINT (10.19668 50.00553),"LINESTRING (9.373378722135374 53.9232825, 10.1...",1.0,NaN
4,5,Bad Lauchstädt,"Bad Lauchstädt, Lindenstraße, Lindenstraße, Kl...",POINT (11.86870 51.38813),Meitingen,"Meitingen, Landkreis Augsburg, Bayern, 86405, ...",POINT (10.85000 48.55000),"LINESTRING (11.8686979 51.3881334, 10.85 48.55)",1.0,NaN


Export shapefile

In [186]:
powerlines_geo.rename(columns = {'direct_line':'geometry'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['#BBPlG2013', 'Start', 'End', 'geometry', 'New', 'Retro'])
powerlines_geo.to_file(f'{path}/data/geodata/powerlines.shp')

/opt/miniconda3/envs/vscode/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
